# Semisupervised learning GAN

In [1]:
%matplotlib inline

import pickle as pkl
import time

import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import tensorflow as tf

SyntaxError: invalid syntax (<ipython-input-1-9062525cd018>, line 8)